### Uses Paper Trade - test money
##### https://alpaca.markets/docs/api-documentation/api-v2/

In [31]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import time, datetime as dt
import pandas as pd, numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import sys
import math

import alpha
import alpaca
import database as db
from populate import download_data
from portfolios import Portfolio
from history import *

DataStore = db.DataStore()

from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.ensemble import AdaBoostRegressor
from sklearn.datasets import make_regression

In [20]:
dates = [dt.datetime(2019,1,1), dt.datetime(2022,12,31)]
sym = 'JPM'
#indicators = ['SMA','OBV', 'AD', 'BBANDS', 'MFI', 'SAR', 'T3', 'MOM', 'MIDPRICE', 'WMA']
indicators = 'all'
history = History(DataStore, sym, dates, indicators=indicators)
indicators = history.indicators
prices = history.prices

indicators.rename(columns={'adjusted close': 'Close'}, inplace=True)

In [21]:
predictors = indicators.iloc[:, 1:]
close_price = indicators.iloc[:, 0]

# predict tomorrow's closing price using today's indicators... 
predictors = predictors[:-1]
close_price = close_price[1:]

In [22]:
predictors.shape

(634, 50)

In [24]:
scaler = StandardScaler()
predictors = scaler.fit_transform(predictors)

#X_train, X_test, y_train, y_test = train_test_split(predictors, close_price, test_size=0.3)
X_train = predictors[:450]
y_train = close_price[:450]

X_test = predictors[451:]
y_test = close_price[451:]

y_train = y_train.to_numpy().astype(float)
y_test = y_test.to_numpy().astype(float)

In [34]:
clf = tree.DecisionTreeRegressor()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_train)
math.sqrt(((y_train - y_pred) ** 2).sum()/y_train.shape[0]) 

0.0

In [35]:
X, y = make_regression(n_features=4, n_informative=2,shuffle=False)
regr = AdaBoostRegressor(n_estimators=100)
regr.fit(X_train, y_train)
regr.score(X_test, y_test)

-0.022968669354906934